In [19]:
import os
import numpy as np
import pandas as pd
from pycaret.regression import *
from datetime import datetime


In [2]:
root="/home/sadagopan/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
samples = pd.read_csv(root + "sample_submission.csv")
#samples[samples['prediction'] == 0].count()[0]

In [20]:
root="/home/sadagopan/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
scols = [col for col in headers if col[0] == 'S']
dcols = [col for col in headers if col[0] == 'D']
rcols = [col for col in headers if col[0] == 'R']
bcols = [col for col in headers if col[0] == 'B']
pcols = [col for col in headers if col[0] == 'P']
labels =pd.read_csv(root+"train_labels.csv")

In [31]:
testdfs = []
i=0
for j in range(12):
    print("t" + str(j) + ".csv")
    testdfs.append(pd.read_csv("t" + str(j) + ".csv"))

    mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')
    maxdate = mindate + pd.offsets.DateOffset(months=j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonthnew'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), left_on=['customer_ID', 'yearmonthnew'], right_on=['customer_ID', 'yearmonth'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'yearmonth_y'], inplace=True)
    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i].drop(columns=['mondiff'], inplace=True)
    testdfs[i].drop(columns=['S_2'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonthnew']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()
    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t" + str(j) +"dtcnt.csv", index=False)
    i+=1

testdfs[0]

t0.csv
t1.csv
t2.csv
t3.csv
t4.csv
t5.csv
t6.csv
t7.csv
t8.csv
t9.csv
t10.csv
t11.csv


,customer_ID,B_1201804,B_10201804,B_11201804,B_12201804,B_13201804,B_14201804,B_15201804,B_16201804,B_17201804,B_18201804,B_19201804,B_2201804,B_20201804,B_21201804,B_22201804,B_23201804,B_24201804,B_25201804,B_26201804,B_27201804,B_28201804,B_29201804,B_3201804,B_30201804,B_31201804,B_32201804,B_33201804,B_36201804,B_37201804,B_38201804,B_39201804,B_4201804,B_40201804,B_41201804,B_42201804,B_5201804,B_6201804,B_7201804,B_8201804,B_9201804,D_102201804,D_103201804,D_104201804,D_105201804,D_106201804,D_107201804,D_108201804,D_109201804,D_110201804,D_111201804,D_112201804,D_113201804,D_114201804,D_115201804,D_116201804,D_117201804,D_118201804,D_119201804,D_120201804,D_121201804,D_122201804,D_123201804,D_124201804,D_125201804,D_126201804,D_127201804,D_128201804,D_129201804,D_130201804,D_131201804,D_132201804,D_133201804,D_134201804,D_135201804,D_136201804,D_137201804,D_138201804,D_139201804,D_140201804,D_141201804,D_142201804,D_143201804,D_144201804,D_145201804,D_39201804,D_41201804,D_42201804,D_43201804,D_44201804,D_45201804,D_46201804,D_47201804,D_48201804,D_49201804,D_50201804,D_51201804,D_52201804,D_53201804,D_54201804,D_55201804,D_56201804,D_58201804,D_59201804,D_60201804,D_61201804,D_62201804,D_63201804,D_64201804,D_65201804,D_66201804,D_68201804,D_69201804,D_70201804,D_71201804,D_72201804,D_73201804,D_74201804,D_75201804,D_76201804,D_77201804,D_78201804,D_79201804,D_80201804,D_81201804,D_82201804,D_83201804,D_84201804,D_86201804,D_87201804,D_88201804,D_89201804,D_91201804,D_92201804,D_93201804,D_94201804,D_96201804,P_2201804,P_3201804,P_4201804,R_1201804,R_10201804,R_11201804,R_12201804,R_13201804,R_14201804,R_15201804,R_16201804,R_17201804,R_18201804,R_19201804,R_2201804,R_20201804,R_21201804,R_22201804,R_23201804,R_24201804,R_25201804,R_26201804,R_27201804,R_28201804,R_3201804,R_4201804,R_5201804,R_6201804,R_7201804,R_8201804,R_9201804,S_11201804,S_12201804,S_13201804,S_15201804,S_16201804,S_17201804,S_18201804,S_19201804,S_20201804,S_22201804,S_23201804,S_24201804,S_25201804,S_26201804,S_27201804,S_3201804,S_5201804,S_6201804,S_7201804,S_8201804,S_9201804,bncnt201804,day_of_month201804,day_of_week201804,day_of_year201804,dncnt201804,pncnt201804,rncnt201804,sncnt201804,yearmonth201804,count
0,0004ef34876259ea444bbe3c15031240a66cc73a7db0ef...,0.001889,0.297802,0.004482,0.415754,0.383141,0.012555,0.007515,0.085317,0.694224,1.006019,0.007691,1.007591,0.007732,0.006630,0.002327,0.007291,0.003918,0.004324,0.004393,0.000222,0.073794,0.002512,0.008544,0.0,1,0.008776,1.004651,0.002278,0.003005,2.0,NaN,0.021716,0.010005,0.006592,0.038483,0.668147,1.986198,0.012818,0.001669,0.006586,0.167325,1.009339,0.954348,0.112588,0.178927,0.336946,0.003503,0.008365,NaN,NaN,1.006251,0.204216,1.000000,0.033055,0.000000,5.000000,0.387421,0.389774,0.000000,0.803222,0.294208,0.009280,1.050578,0.001728,1.0,1.001007,1.005469,1.005156,0.001651,0.005311,0.156404,0.006540,0.575816,0.057688,0.172528,0.044493,0.195624,0.008753,0.005718,0.009689,0.376373,0.003142,0.000315,0.005288,0.037497,0.006650,0.155319,0.062525,0.003414,0.154393,0.356929,0.816518,0.022508,0.170376,0.200714,0.007389,0.399255,0.072799,1.006216,0.052568,0.163459,0.001409,0.355019,0.015797,0.024192,0.536743,XM,O,0.000884,1.0,4.000000,0.003123,0.000303,1.396719,0.006002,0.179424,0.000266,0.009328,2.561972,0.500940,0.001557,0.009377,0.609942,0.009614,0.363272,0.009026,0.002267,0.003989,NaN,0.007812,0.005231,0.009414,0.001902,0.000959,1.001530,0.007815,0.969106,0.620409,0.008506,0.003711,0.001544,0.000794,1.009268,0.008098,0.004756,0.005093,0.006197,0.009613,0.002902,0.005261,0.003198,0.008604,0.007711,0.006710,0.002911,0.003791,0.003491,0.087197,1.008731,0.008815,0.009339,0.005306,0.006778,0.005756,0.004741,0.000430,0.232349,0.844160,0.186833,0.045782,0.306733,0.005376,0.065739,0.002359,0.001121,0.003656,0.864283,0.139612,0.876456,0.971742,1.989535,0.671630,0.104756,0.002875,0.006084,0.086756,0.331656,0.004853,93,24,2,114,1062,0,37,0,201904,1
1,00085c285868ee0995a0d47eb096a229720abbf0fc8e3a

In [ ]:
predsrange = []
predictions=pd.DataFrame()
for i in range(1):
    try:
        model = load_model(root+"/models/" + "x" + str(i) + "stackedreg")
    except:
        try:
            model = load_model(root+"/models/" + "lightgbm-x" + str(i) + "reg")
        except:
            try:
                model = load_model(root+"/models/" + "knn-x" + str(i) + "reg")
            except:
                try:
                    model = load_model(root+"/models/" + "rf-x" + str(i) + "reg")
                except:
                    model = load_model(root+"/models/" + "et-x" + str(i) + "reg")

    testdf = (pd.read_csv("x" + str(i) + "dtcnt.csv"))
    preds = predict_model(model, data=testdf, verbose=True)
    preds.to_csv(root+"/test/x"+ str(i) + ".dtcnt" + ".preds", index=False)
    print((preds['Label'].min(), preds['Label'].max()))
    if len(predsrange) <= i:
        predsrange.append((preds['Label'].min(), preds['Label'].max()))
    else:
        predsrange[i] = (preds['Label'].min(), preds['Label'].max())
    print("t" + str(i) + "dtcnt.csv")
    testdf = (pd.read_csv("t" + str(i) + "dtcnt.csv"))
    preds = predict_model(model, data=testdf, verbose=True)
#    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
    preds[['customer_ID', 'Label']].to_csv(root+"/test/t"+ str(i) + ".dtcnt2" + ".preds", header=False, index=False)

print((predsrange[i-1].min(), predsrange[i-1].max()))
if len(predsrange) <= i:
    predsrange.append((preds['Label'].min(), preds['Label'].max()))
else:
    predsrange[i] = (preds['Label'].min(), preds['Label'].max())

#for i in range(41):
#    print("t12-" + str(i) + "dtcnt.csv")
#    testdf = (pd.read_csv("t12-" + str(i) + "dtcnt.csv"))
#    preds = predict_model(model, data=testdf, verbose=True)
#    preds[['customer_ID', 'Label']].to_csv(root+"/test/t12-"+ str(i) + ".dtcnt2" + ".preds", header=False, index=False)

predsrange

In [16]:
i=12
print(len(predsrange), predsrange[i], predsrange[i-2])
print((predsrange[i-1][0], predsrange[i-1][1]))
if len(predsrange) <= i:
    predsrange.append((predsrange[i-2][0], predsrange[i-2][1]))
else:
    predsrange[i] = (predsrange[i-2][0], predsrange[i-2][1])

model = load_model(root+"/models/" + "x10stackedreg")
for i in range(41):
    print("t12-" + str(i) + "dtcnt.csv")
    testdf = (pd.read_csv("t12-" + str(i) + "dtcnt.csv"))
    preds = predict_model(model, data=testdf, verbose=True)
    preds[['customer_ID', 'Label']].to_csv(root+"/test/t12-"+ str(i) + ".dtcnt" + ".preds", header=False, index=False)

predsrange

13 (-0.043259817614827556, 3.156651145188592) (-0.060757598058207796, 1.4092763591569941)
(-0.060757598058207796, 1.4092763591569941)
Transformation Pipeline and Model Successfully Loaded
t12-0dtcnt.csv
t12-1dtcnt.csv
t12-2dtcnt.csv
t12-3dtcnt.csv
t12-4dtcnt.csv
t12-5dtcnt.csv
t12-6dtcnt.csv
t12-7dtcnt.csv
t12-8dtcnt.csv
t12-9dtcnt.csv
t12-10dtcnt.csv
t12-11dtcnt.csv
t12-12dtcnt.csv
t12-13dtcnt.csv
t12-14dtcnt.csv
t12-15dtcnt.csv
t12-16dtcnt.csv
t12-17dtcnt.csv
t12-18dtcnt.csv
t12-19dtcnt.csv
t12-20dtcnt.csv
t12-21dtcnt.csv
t12-22dtcnt.csv
t12-23dtcnt.csv
t12-24dtcnt.csv
t12-25dtcnt.csv
t12-26dtcnt.csv
t12-27dtcnt.csv
t12-28dtcnt.csv
t12-29dtcnt.csv
t12-30dtcnt.csv
t12-31dtcnt.csv
t12-32dtcnt.csv
t12-33dtcnt.csv
t12-34dtcnt.csv
t12-35dtcnt.csv
t12-36dtcnt.csv
t12-37dtcnt.csv
t12-38dtcnt.csv
t12-39dtcnt.csv
t12-40dtcnt.csv


[(-0.043259817614827556, 3.156651145188592),
 (-0.09766120626214893, 1.0425727165242535),
 (0.037550327929235294, 0.5209467047918106),
 (-0.12003841159122608, 2.414134160694392),
 (-0.1074236883168338, 12.45373702699337),
 (-0.10433507347204962, 9.345122891610988),
 (-0.11432391499602979, 2.9051420718305323),
 (-0.05951777175425432, 1.2272600096409039),
 (-0.0481976348460309, 1.2053768725904432),
 (-0.059356542062170226, 1.2100175224174872),
 (-0.060757598058207796, 1.4092763591569941),
 (-0.060757598058207796, 1.4092763591569941),
 (-0.060757598058207796, 1.4092763591569941)]

In [62]:
preds = []
for r, d, f in os.walk(root+"test"):
        f.sort()
#    if r == "/amex/test":
        for file in f:
            if ".dtcnt.preds" in file :
                preds.append(os.path.join(r, file))
preds[len(preds)-1], len(preds)

('/home/sadagopan/amex-default-prediction/test/t9.dtcnt.preds', 11)

In [10]:
preddf= pd.DataFrame()
for i in range(12):# len(preds)):
    preds = pd.read_csv(root+"/test/x"+ str(i) + ".dtcnt" + ".preds", names=['customer_ID', 'Label'])
    rangemin = preds['Label'].min()
    rangemax = preds['Label'].max()
    preds = pd.read_csv(root+"/test/t"+ str(i) + ".dtcnt" + ".preds", names=['customer_ID', 'Label'])
    rangemin = min(df['Label'].min(), rangemin)
    rangemax = max(df['Label'].max(), rangemax)
    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)

for i in range(41):# len(preds)):
    preds = pd.read_csv(root+"/test/x11.dtcnt.preds", names=['customer_ID', 'Label'])
    rangemin = preds['Label'].min()
    rangemax = preds['Label'].max()
    preds = pd.read_csv(root+"/test/t12-"+ str(i) + ".dtcnt" + ".preds", names=['customer_ID', 'Label'])
    rangemin = min(df['Label'].min(), rangemin)
    rangemax = max(df['Label'].max(), rangemax)
    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)


-0.0486579768431089 0.9999857731386508 actual: -0.0486579768431089 0.9999857731386508
-0.0955287041282816 1.0192627445073486 actual: -0.0955287041282816 1.0192627445073486
0.0415255393846938 0.4269033367568553 actual: 0.0415255393846938 0.4269033367568553
-0.0726377617688687 1.348070583110943 actual: -0.0726377617688687 1.348070583110943
-0.0550439845779959 1.9432495227322384 actual: -0.0550439845779959 1.9432495227322384
-0.0548439802742282 2.21670423770394 actual: -0.0548439802742282 2.21670423770394
-0.0683035687566147 1.948230277559507 actual: -0.0683035687566147 1.948230277559507
-0.0645470731428394 1.4470256669316997 actual: -0.0645470731428394 1.4470256669316997
-0.0711339511153565 1.2819812879361745 actual: -0.0711339511153565 1.2819812879361745
-0.0639273287230182 1.2553079355194252 actual: -0.0639273287230182 1.2553079355194252
-0.0500451394749792 1.4301358438582752 actual: -0.0500451394749792 1.4301358438582752
-0.0432598176148275 3.156651145188592 actual: -0.043259817614827

In [5]:
preddf['Label'].min(), preddf['Label'].max()

(0.0, 1.0)

In [6]:
preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()
valids = pd.merge(samples, preddf, on='customer_ID', how='inner')
valids.rename(columns={'prediction': 'target'}, inplace=True)
valids.rename(columns={'Label': 'prediction'}, inplace=True)

valids['prediction'] = ( (valids[['prediction']] - valids[['prediction']].min()) / (valids[['prediction']].max() - valids[['prediction']].min()) ) * (1 - 0) + 0

print("matching 1's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 1)].count()[0], valids[(valids['target'] == 1)].count()[0])
print("matching 0's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 0)].count()[0], valids[(valids['target'] == 0)].count()[0])
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 1's 0 0
matching 0's 16319 16327
matching 16319 total 16327 % 0.999510014087095


In [ ]:
valids[valids['prediction']>= 0.8497913915].count()/793212
#valids['prediction'].min(), valids['prediction'].max()
#pd.DataFrame(
valids.groupby(['prediction', 'target']).count().sort_values(by=["prediction"], ascending=False).head(50)

In [31]:
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 736896 total 924621 % 0.7969708669822554


In [ ]:
valids[(valids['prediction'] <= 1)].sort_values('prediction', ascending=False)

In [19]:
#!mv /amex/submission.csv submission11.csv

valids[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)

In [154]:
800000/924621
38716/(38716+119855)

0.24415561483499504

In [4]:
#preddf= pd.DataFrame()
#for i in range(78):
#    preddf = pd.concat([preddf, pd.read_csv(preds[i], names=['customer_ID', 'Label'], header=None)], ignore_index=True, axis=0)

In [5]:
#preddf['Label'] = preddf['Label'].apply(lambda x: abs(x))
#preddf['Label'] = (preddf['Label'] / preddf['Label'].max())
#preddf['Label'] = preddf['Label'].apply(lambda x: 1 if x>1 else x)
#preddf['Label'] = preddf['Label'].apply(lambda x: 0 if x<0 else x)

#preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()

#pd.DataFrame(preddf)
#preddf['Label'] = preddf.Label.astype("int")
#preddf.head(1)

In [10]:
valids['prediction'].agg(['min', 'max'])
#731000/924621

min    0.0
max    1.0
Name: prediction, dtype: float64

In [ ]:
#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] = valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] * 0.9

#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction']

df = (pd.concat([valids[['customer_ID', 'prediction']].rename(columns={'prediction':'target'}), valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)
#df = (pd.concat([valids[['customer_ID', 'target']], valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)

#df['correcpred'] = df['prediction'].apply(lambda x: 1 if x> 1 else (0 if x < 0 else x))

df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)

df['random'] = (df['weight'] / df['weight'].sum()).cumsum()

total_pos = (df['target'] * df['weight']).sum()

df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()

df['lorentz'] = df['cum_pos_found'] / total_pos

df['gini'] = (df['lorentz'] - df['random']) * df['weight']

#(df['correcpred'] * df['weight']).sum(), (df['prediction'] * df['weight']).sum()
df['gini'].sum()


#1782137.8168000695/303412.06746732915

#valids[valids['prediction']>1].max()
df#[df['target']>0.9]

In [ ]:
int(0.04 * df['weight'].sum())
(df['target'] * df['weight']).sum()

In [76]:
tst = pd.read_csv("/amex/test/testaa", names=headers)

In [77]:
tst={}

In [ ]:
#traindf = pd.read_csv("/amex/trainaa", names=headers)

traindf [['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]

pd.DataFrame(traindf ['D_120'].unique())


In [8]:
#x11preds=pd.read_csv(root+"/test/x11.dtcnt.preds")
x11preds[['customer_ID', 'target', 'Label']]

print("matching 1's", x11preds[(x11preds['target'] == round(x11preds['Label'])) & (x11preds['target'] == 1)].count()[0], x11preds[(x11preds['target'] == 1)].count()[0])
print("matching 0's", x11preds[(x11preds['target'] == round(x11preds['Label'])) & (x11preds['target'] == 0)].count()[0], x11preds[(x11preds['target'] == 0)].count()[0])
print("matching", x11preds[x11preds['target'] == round(x11preds['Label'])].count()[0], "total", x11preds.count()[0], "%", x11preds[x11preds['target'] == round(x11preds['Label'])].count()[0]/x11preds.count()[0])

matching 1's 3619 4136
matching 0's 6423 6487
matching 10042 total 10623 % 0.9453073519721359


In [ ]:
for column in [col for col in t11.columns if col[len(col)-6:] == '201903']:
    t11.drop(columns=[column], inplace=True)
t11

In [101]:
t11.to_csv(root+"x11dtcnt.csv", index=False)